# Odia BPE Tokenizer

This notebook demonstrates the basic usage of the OdiaBPETokenizer class.



**complete Unicode range for the Odia (ଓଡ଼ିଆ) script**, as defined by the Unicode Consortium.

---

### 📘 **Odia Unicode Block**

| Property              | Value             |
| --------------------- | ----------------- |
| **Block name**        | Odia              |
| **Unicode range**     | `U+0B00 – U+0B7F` |
| **Code points count** | 128 (0x80)        |
| **Script**            | Odia              |
| **Direction**         | Left-to-Right     |

---

### 🔤 **Detailed Character Categories**

| Category                              | Unicode Range                               | Description                        | Example                     |
| ------------------------------------- | ------------------------------------------- | ---------------------------------- | --------------------------- |
| **Independent vowels**                | U+0B05–U+0B14                               | ଅ, ଆ, ଇ, ଈ, ଉ, ଊ, ଋ, ୠ, ଏ, ଐ, ଓ, ଔ | ଅ, ଆ                        |
| **Consonants**                        | U+0B15–U+0B39                               | କ, ଖ, ଗ, ଘ ... ହ                   | କ, ଗ                        |
| **Dependent vowel signs (matras)**    | U+0B3E–U+0B44, U+0B47–U+0B48, U+0B4B–U+0B4C | ୀ, ୁ, ୂ, ୃ, ୈ                      | ୀ, ୂ                        |
| **Virama (Halant)**                   | U+0B4D                                      | ୍                                  | used for consonant clusters |
| **Anusvara / Visarga / Chandrabindu** | U+0B01–U+0B03                               | ଁ, ଂ, ଃ                            | ଂ                           |
| **Digits (Odia numerals)**            | U+0B66–U+0B6F                               | ୦–୯                                | ୧, ୨                        |
| **Punctuation & signs**               | U+0B70–U+0B77                               | ୰, ୱ, ୲                            | ୱ                           |
| **Odia sign Nukta**                   | U+0B3C                                      | ଡ଼ → ଡ + Nukta                      | ଡ଼                           |
| **Odia sign Avagraha**                | U+0B3D                                      | ଽ                                  | Used in Sanskrit contexts   |


`ODIA_SPLIT_PATTERN = r"""[\u0B00-\u0B7F]+|\d+|[^\s\u0B00-\u0B7F\d]+| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""`

## 1. Import and Setup


In [1]:
from odia_bpe_tokenizer import OdiaBPETokenizer, ODIA_SPLIT_PATTERN
from dataset import load_odia_dataset

## 2. Load Dataset and Prepare Training Data


In [2]:
# Load dataset
ds = load_odia_dataset('./data')

# Use a subset for training
num_samples = 500000
training_text = ' '.join(ds['train']['text'][:num_samples])

print(f"Training text size: {len(training_text):,} characters")
print(f"Sample: {training_text[:100]}...")


Resolving data files:   0%|          | 0/25 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset loaded successfully!
Dataset structure: DatasetDict({
    train: Dataset({
        features: ['text', 'source'],
        num_rows: 5941930
    })
})
Training text size: 49,299,688 characters
Sample: ଆଇସିସି ବିଶ୍ବକପରେ ସେ ଅଂଶ ଗ୍ରହଣ କରିଥିବାରୁ ସେ ନିଜକୁ ଭାଗ୍ୟଶାଳୀ ମନେ କରୁଛନ୍ତି । କରୀନା କପୂର ହେଉଛନ୍ତି ଭାରତୀୟ...


## 3. Train Tokenizer


In [3]:
# Create and train tokenizer
tokenizer = OdiaBPETokenizer(pattern=ODIA_SPLIT_PATTERN)

print("Training tokenizer...")
stats = tokenizer.train(training_text, vocab_size=4000, verbose=True)


Training tokenizer...
Text split into 14827345 chunks
merge 1/3744: (224, 172) -> 256 (�) had 31586333 occurrences
merge 2/3744: (224, 173) -> 257 (�) had 9102226 occurrences
merge 3/3744: (256, 190) -> 258 (ା) had 3680563 occurrences
merge 4/3744: (257, 141) -> 259 (୍) had 3621787 occurrences
merge 5/3744: (256, 176) -> 260 (ର) had 3412710 occurrences
merge 6/3744: (256, 191) -> 261 (ି) had 3381404 occurrences
merge 7/3744: (259, 256) -> 262 (୍�) had 2288435 occurrences
merge 8/3744: (261, 256) -> 263 (ି�) had 2233023 occurrences
merge 9/3744: (258, 256) -> 264 (ା�) had 2157223 occurrences
merge 10/3744: (257, 135) -> 265 (େ) had 1808806 occurrences
merge 11/3744: (256, 149) -> 266 (କ) had 1349971 occurrences
merge 12/3744: (257, 129) -> 267 (ୁ) had 1148230 occurrences
merge 13/3744: (256, 168) -> 268 (ନ) had 1094714 occurrences
merge 14/3744: (256, 184) -> 269 (ସ) had 1040635 occurrences
merge 15/3744: (256, 172) -> 270 (ବ) had 1031023 occurrences
merge 16/3744: (256, 170) -> 271 (ପ)

## 4. Test Encoding and Decoding


In [4]:
# Test on example texts
test_text = "କାର୍ତ୍ତିକ ପୂର୍ଣ୍ଣିମା ଉପଲକ୍ଷେ ସମସ୍ତଙ୍କୁ ହାର୍ଦ୍ଦିକ ଶୁଭେଚ୍ଛା ଆଉ ଶୁଭକାମନା। ବନ୍ଦେ ଉତ୍କଳ ଜନନୀ"

# Encode
encoded = tokenizer.encode(test_text)
print(f"Original: {test_text}")
print(f"Encoded:  {encoded}")
print(f"Tokens:   {len(encoded)}")

# Decode
decoded = tokenizer.decode(encoded)
print(f"Decoded:  {decoded}")
print(f"Match:    {test_text == decoded}")

# Compression stats
comp_stats = tokenizer.get_compression_stats(test_text)
print(f"\nCompression ratio: {comp_stats['compression_ratio']:.2f}x")
print(f"Bytes per token:   {comp_stats['bytes_per_token']:.2f}")


Original: କାର୍ତ୍ତିକ ପୂର୍ଣ୍ଣିମା ଉପଲକ୍ଷେ ସମସ୍ତଙ୍କୁ ହାର୍ଦ୍ଦିକ ଶୁଭେଚ୍ଛା ଆଉ ଶୁଭକାମନା। ବନ୍ଦେ ଉତ୍କଳ ଜନନୀ
Encoded:  [351, 482, 306, 32, 1187, 3970, 32, 552, 638, 265, 32, 3564, 32, 893, 514, 514, 306, 32, 1963, 3240, 258, 32, 928, 32, 1963, 866, 413, 224, 165, 164, 32, 951, 265, 32, 3186, 310, 32, 706, 681]
Tokens:   39
Decoded:  କାର୍ତ୍ତିକ ପୂର୍ଣ୍ଣିମା ଉପଲକ୍ଷେ ସମସ୍ତଙ୍କୁ ହାର୍ଦ୍ଦିକ ଶୁଭେଚ୍ଛା ଆଉ ଶୁଭକାମନା। ବନ୍ଦେ ଉତ୍କଳ ଜନନୀ
Match:    True

Compression ratio: 6.18x
Bytes per token:   6.18


## 5. Save and Load Tokenizer


In [5]:
# Save tokenizer
tokenizer.save("odia_bpe_tokenizer")

# Load it back
loaded_tokenizer = OdiaBPETokenizer()
loaded_tokenizer.load("odia_bpe_tokenizer.model")

# Test that it works
test = "ଓଡ଼ିଆ"
original_encoded = tokenizer.encode(test)
loaded_encoded = loaded_tokenizer.encode(test)

print(f"Original: {original_encoded}")
print(f"Loaded:   {loaded_encoded}")
print(f"Match:    {original_encoded == loaded_encoded}")


Tokenizer saved to odia_bpe_tokenizer.model and odia_bpe_tokenizer.vocab
Tokenizer loaded from odia_bpe_tokenizer.model
Vocabulary size: 4000
Number of merges: 3744
Original: [2060]
Loaded:   [2060]
Match:    True
